In [1]:
import csv
import torch
import random
import pandas as pd
import numpy as np
from transformers import XLMRobertaTokenizerFast
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaForTokenClassification

## DATA PREPARATION

In [2]:
csv_file_path = 'Labeled_dataset_de.csv'

sentences = {}
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)
    for row in csv_reader:
        sentence_id, token, label = row
        if sentence_id not in sentences:
            sentences[sentence_id] = {'tokens': [], 'labels': []}
        sentences[sentence_id]['tokens'].append(token)
        sentences[sentence_id]['labels'].append(label)

result = []
for sentence_id, data in sentences.items():
    words = data['tokens']
    labels = data['labels']
    result.append((words, labels))

for sentence in result:
    print(sentence)

(['es', 'gibt', 'leider', 'einen', 'haufen', 'perverse', 'wixer', 'auf', 'dieser', 'welt', 'und', 'in', 'unseren', 'städten', ',', 'dass', 'man', 'sich', 'dann', 'so', 'über', 'männer', 'aufregen', 'muss', 'die', 'breiter', 'sitzen', '.'], ['n', 'n', 'n', 'n', 'n', 'n', 'B-B', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n'])
(['hier', 'macht', 'sich', 'deutschalnd', 'in', 'verfassungswidriger', 'weise', 'zum', 'mittäter', 'an', 'solche', 'taten', '!'], ['n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n'])
(['das', 'ist', 'so', 'dumm', '!'], ['n', 'n', 'n', 'B-B', 'n'])
(['mein', 'freund', 'hat', 'ein', 'tolles', 'bild', 'von', 'meinem', 'auto', 'gemacht', '.'], ['n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n'])
(['der', 'witz', 'liegt', 'mir', 'auf', 'der', 'zunge', '.'], ['n', 'n', 'n', 'n', 'n', 'n', 'n', 'n'])
(['wenn', 'ich', 'einen', 'asiaten', 'auf', 'der', 'straße', 'oder', 'auf', 'dem', 'schulhof', 'als

In [3]:
csv_file_path = 'Labeled_dataset_en.csv'

sentences = {}
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)
    for row in csv_reader:
        sentence_id, token, label = row
        if sentence_id not in sentences:
            sentences[sentence_id] = {'tokens': [], 'labels': []}
        sentences[sentence_id]['tokens'].append(token)
        sentences[sentence_id]['labels'].append(label)

result_2 = []
for sentence_id, data in sentences.items():
    words = data['tokens']
    labels = data['labels']
    result_2.append((words, labels))

for sentence in result_2:
    print(sentence)

(['her', 'wet', ',', 'quivering', 'quim', 'belied', 'her', 'apparent', 'disinterest', 'in', 'me', '.'], ['n', 'n', 'n', 'n', 'B-B', 'n', 'n', 'n', 'n', 'n', 'n', 'n'])
(['you', 'spics', 'are', 'worthless', ',', 'why', 'ca', "n't", 'you', 'learn', 'english', 'like', 'everyone', 'else', '?'], ['n', 'B-B', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n'])
(['too', 'bad', 'that', "'s", 'exactly', 'why', 'he', 'ca', "n't", 'win', '.'], ['n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n'])
(['so', 'far', 'she', 'does', '.'], ['n', 'n', 'n', 'n', 'n'])
(['i', 'mean', 'if', 'a', 'minute', 'passed', 'between', 'all', 'that', 'then', 'holy', 'shit', 'i', "'d", 'have', 'been', 'surprised', '.'], ['n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'B-B', 'B', 'n', 'n', 'n', 'n', 'n', 'n'])
(['what', 'a', 'clusterfuck', 'of', 'awful', 'qualities', '.'], ['n', 'n', 'B-B', 'n', 'n', 'n', 'n'])
(['i', "'ve", 'been', 'trying', 'to', 'get', 'a', 'crazy', 'win', 'like', 'that', 'but', '

In [4]:
train_data_de, test_data_de = train_test_split(result, test_size=0.2, random_state=42)

val_data_de, test_data_de = train_test_split(test_data_de, test_size=0.5, random_state=42)

print(f"Training set length: {len(train_data_de)}")
print(f"Validation set length: {len(val_data_de)}")
print(f"Test set length: {len(test_data_de)}")

Training set length: 468
Validation set length: 58
Test set length: 59


In [5]:
train_data_en, test_data_en = train_test_split(result_2, test_size=0.2, random_state=42)

val_data_en, test_data_en = train_test_split(test_data_en, test_size=0.5, random_state=42)

print(f"Training set length: {len(train_data_en)}")
print(f"Validation set length: {len(val_data_en)}")
print(f"Test set length: {len(test_data_en)}")

Training set length: 980
Validation set length: 122
Test set length: 123


In [6]:
train_data = train_data_de + train_data_en
val_data = val_data_de + val_data_en
test_data = test_data_de + test_data_en

In [7]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

1448
180
182


In [8]:
print(train_data)

[(['die', 'hure', 'wird', 'sich', 'nie', 'wieder', 'an', 'meinen', 'freund', 'ranmachen', '.'], ['n', 'B-B', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n']), (['hey', ',', 'arschgesicht', '!'], ['n', 'n', 'B-B', 'n']), (['ach', 'ja', 'ich', 'blödmann', ',', 'steht', 'doch', 'im', 'artikel', ',', 'weil', 'so', 'viele', 'fahrräder', 'auf', 'den', 'straßen', 'unterwegs', 'sind', '.'], ['n', 'n', 'n', 'B-B', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n']), (['außer', 'mit', 'nörgelnder', 'stimme', 'dummes', 'zeug', 'zu', 'quatschen', 'und', 'wie', 'ein', 'idiot', 'aus', 'zu', 'sehen', 'hat', 'dieser', 'typ', 'keine', 'weitere', 'qualitäten', '.'], ['n', 'n', 'n', 'n', 'B-B', 'n', 'n', 'n', 'n', 'n', 'n', 'B-B', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n', 'n']), (['die', 'sind', 'auch', 'nur', 'ein', 'haufen', 'huren', '.'], ['n', 'n', 'n', 'n', 'n', 'n', 'B-B', 'n']), (['es', 'hat', 'noch', 'nie', 'auf', 'den', 'fußweg', 'gekackt', ',', 'und', 'erst', 'kürzl

In [9]:
gold_set_for_validation_de = set(word for (words, labels) in val_data_de for word, label in zip(words, labels) if label == 'B-B' or label == 'B')

df_gold_val_de = pd.DataFrame({'terms': list(gold_set_for_validation_de)})
print(df_gold_val_de)

            terms
0       shitstorm
1        feigling
2       flittchen
3          transe
4       arschloch
5      blödmänner
6    fotzenlecker
7           nutte
8          kaffer
9      hackfresse
10    dumpfbacken
11         nippel
12    wurmwichser
13            ins
14    hodenbeißer
15    arschficker
16  hosenscheißer
17    volltrottel
18          pöbel
19            kuh
20       abschaum
21       mistkerl
22         dummen
23         ficken
24          blöde
25     scheißkerl
26   dreckskerlen
27       blödmann
28      scheissen
29           möse
30           knie
31          fotze
32           fick
33       anschiss
34   rattenscharf
35        scheiße
36  arschkriecher
37        rechten
38     hirnrissig
39        schwein
40            sau
41         bumsen
42      arschfick
43     beschissen
44   vollscheißen
45    vollscheißt
46           dich
47      vollidiot


In [ ]:
gold_set_for_validation_en = set(word for (words, labels) in val_data_en for word, label in zip(words, labels) if label == 'B-B' or label == 'B')

df_gold_val_en = pd.DataFrame({'terms': list(gold_set_for_validation_en)})
print(df_gold_val_en)

        terms
0    crippled
1        gash
2        cock
3     scumbag
4      idiots
..        ...
98    bitches
99   shitface
100   redneck
101      what
102      piss

[103 rows x 1 columns]


In [11]:
gold_set_for_test_de = set(word for (words, labels) in test_data_de for word, label in zip(words, labels) if label == 'B-B' or label == 'B')

df_gold_test_de = pd.DataFrame({'terms': list(gold_set_for_test_de)})
print(df_gold_test_de)

              terms
0       drecksfotze
1       vollidioten
2      vollscheißen
3            muschi
4            kanake
5             idiot
6         arschloch
7            kaffer
8             arsch
9        hackfresse
10               im
11     gottverdammt
12      morgenlatte
13        drecksack
14       schwuchtel
15       scheißegal
16      arschficker
17      volltrottel
18             dumm
19           pimmel
20           pisser
21         schlampe
22         großmaul
23           lecken
24         asoziale
25            stück
26         dummkopf
27        scheissen
28          trottel
29     schweinehund
30            fotze
31             fick
32       fickfehler
33          scheiße
34       hirnrissig
35       arschgeige
36        hurensohn
37            dumme
38              sau
39           deppen
40     flachwichser
41             homo
42           bumsen
43       volldeppen
44             depp
45       beschissen
46          schwanz
47  schwanzlutscher


In [ ]:
gold_set_for_test_en = set(word for (words, labels) in test_data_en for word, label in zip(words, labels) if label == 'B-B' or label == 'B')

df_gold_test_en = pd.DataFrame({'terms': list(gold_set_for_test_en)})
print(df_gold_test_en)

        terms
0      fuckup
1      fucked
2     scumbag
3         fap
4       sluts
..        ...
92    banging
93    dipshit
94       bang
95  shithouse
96       piss

[97 rows x 1 columns]


In [13]:
gold_set_for_test = gold_set_for_test_en | gold_set_for_test_de
print(gold_set_for_test)
gold_set_for_validation = gold_set_for_validation_en | gold_set_for_validation_de
print(gold_set_for_validation)

{'morons', 'shithouse', 'vollidioten', 'muschi', 'retard', 'circle', 'dipshit', 'kanake', 'arses', 'idiot', 'badass', 'shits', 'arschloch', 'bumfuck', 'shitty', 'arsch', 'hackfresse', 'contest', 'morgenlatte', 'wanker', 'sluts', 'drecksack', 'schwuchtel', 'fuckin', 'chimps', 'fairy', 'fuckwit', 'cooch', 'dumm', 'nads', 'brain', 'pathetic', 'pisser', 'pecker', 'misogynistic', 'turds', 'shithead', 'dirty', 'schizo', 'shitlist', 'shitfaced', 'pissing', 'dummkopf', 'cocks', 'dyke', 'faggot', 'candy-ass', 'trottel', 'cripple', 'tramp', 'fap', 'fuckery', 'schweinehund', 'fick', 'fickfehler', 'sandwich', 'lardass', 'cum', 'fart', 'hirnrissig', 'faggotry', 'arschgeige', 'hurensohn', 'sau', 'deppen', 'flachwichser', 'holyshit', 'bumsen', 'pile', 'fucked', 'cunt', 'shitass', 'piss', 'volldeppen', 'bang', 'shit', 'depp', 'beschissen', 'dogshit', 'schwanz', 'fugly', 'shut', 'scum', 'drecksfotze', 'schwanzlutscher', 'finger', 'big-ass', 'retards', 'turd', 'piss-up', 'pissant', 'kaffer', 'sub-human'

In [ ]:
df_gold_test = pd.concat([df_gold_test_de, df_gold_test_en])
print(df_gold_test)

df_gold_val = pd.concat([df_gold_val_de, df_gold_val_en])
print(df_gold_val)

          terms
0        pimmel
1   drecksfotze
2          fick
3         idiot
4         fotze
..          ...
92      banging
93      dipshit
94         bang
95    shithouse
96         piss

[145 rows x 1 columns]
             terms
0    hosenscheißer
1    arschkriecher
2            blöde
3             fick
4            fotze
..             ...
98         bitches
99        shitface
100        redneck
101           what
102           piss

[151 rows x 1 columns]


In [15]:
train_tags=[tup[1] for tup in train_data]
train_texts=[tup[0] for tup in train_data]

val_tags=[tup[1] for tup in val_data]
val_texts=[tup[0] for tup in val_data]

test_tags=[tup[1] for tup in test_data]
test_texts=[tup[0] for tup in test_data]

In [17]:
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

In [18]:
label_list=["n", "B-B", "B"]
label_to_id = {l: i for i, l in enumerate(label_list)}
num_labels=len(label_list)

def tokenize_and_align_labels(texts, tags):
  tokenized_inputs = tokenizer(
      texts,
      padding=True,
      truncation=True,
      is_split_into_words=True,
  )
  labels = []
  for i, label in enumerate(tags):
      word_ids = tokenized_inputs.word_ids(batch_index=i)
      previous_word_idx = None
      label_ids = []
      for word_idx in word_ids:
          if word_idx is None:
              label_ids.append(-100)
          elif word_idx != previous_word_idx:
              label_ids.append(label_to_id[label[word_idx]])
          else:
              label_ids.append(-100)
          previous_word_idx = word_idx

      labels.append(label_ids)
  tokenized_inputs["labels"] = labels
  return tokenized_inputs

In [19]:
train_input_and_labels = tokenize_and_align_labels(train_texts, train_tags)

val_input_and_labels = tokenize_and_align_labels(val_texts, val_tags)

test_input_and_labels = tokenize_and_align_labels(test_texts, test_tags)

In [20]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [21]:
train_dataset = Dataset(train_input_and_labels, train_input_and_labels["labels"])

val_dataset = Dataset(val_input_and_labels, val_input_and_labels["labels"])

test_dataset = Dataset(test_input_and_labels, test_input_and_labels["labels"])

In [22]:
def extract_terms(token_predictions, val_tokens):
    extracted_terms = set()
    for i in range(len(token_predictions)):
        pred = token_predictions[i]
        txt = val_tokens[i]
        term = ""
        inside_term = False
        for j in range(len(pred)):
            if pred[j] == "B-B" or pred[j] == "B":
                if inside_term:
                    extracted_terms.add(term)
                    term = ""
                    inside_term = False
                term += txt[j]
                inside_term = True
            elif pred[j] == "n":
                if inside_term:
                    extracted_terms.add(term)
                    term = ""
                    inside_term = False
        if inside_term:
            extracted_terms.add(term)
    return extracted_terms

In [23]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    extracted_terms=extract_terms(true_predictions, val_texts)
    extracted_terms = set([item.lower() for item in extracted_terms])
    gold_set=gold_set_for_validation

    true_pos=extracted_terms.intersection(gold_set)
    recall=len(true_pos)/len(gold_set)
    precision=len(true_pos)/len(extracted_terms)

    return {
        "precision": precision,
        "recall": recall,
        "f1": 2*(precision*recall)/(precision+recall),
    }

## FINE-TUNING

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=0,
    weight_decay=0.004965345054473236,
    learning_rate=7.501154180654612e-05,
    logging_dir='./logs',
    eval_strategy= "steps",
    eval_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

In [ ]:
model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=num_labels)

In [ ]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1
100,No log,0.093317,0.721053,0.913333,0.805882
200,No log,0.063833,0.818713,0.933333,0.872274
300,No log,0.057581,0.856250,0.913333,0.883871
400,No log,0.049946,0.915033,0.933333,0.924092
500,0.066100,0.053715,0.921053,0.933333,0.927152


TrainOutput(global_step=546, training_loss=0.060830899101473906, metrics={'train_runtime': 206.6338, 'train_samples_per_second': 42.045, 'train_steps_per_second': 2.642, 'total_flos': 345845715329088.0, 'train_loss': 0.060830899101473906, 'epoch': 6.0})

## TEST SET

In [24]:
def compute_metrics_test(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    extracted_terms=extract_terms(true_predictions, test_texts)
    extracted_terms = set([item.lower() for item in extracted_terms])
    gold_set=gold_set_for_test

    true_pos=extracted_terms.intersection(gold_set)
    recall=len(true_pos)/len(gold_set)
    precision=len(true_pos)/len(extracted_terms)

    return {
        "precision": precision,
        "recall": recall,
        "f1": 2*(precision*recall)/(precision+recall),
    }

In [27]:
training_args = TrainingArguments(
    output_dir='./results',
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=10,
    do_eval=True
)

In [28]:
trainer = Trainer(
        model=model,
        args=training_args,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics_test,
    )

In [ ]:
eval_results = trainer.evaluate()

In [30]:
test_predictions, test_labels, test_metrics = trainer.predict(test_dataset)
test_predictions = np.argmax(test_predictions, axis=2)
true_test_predictions = [
    [label_list[p] for (p, l) in zip(test_prediction, test_label) if l != -100]
    for test_prediction, test_label in zip(test_predictions, test_labels)
]

In [33]:
i=10
print('{:>10}  {:>10}  {:>10}'.format("Text", "Label", "Prediction"))
for j in range(len(true_test_predictions[i])):
  print('{:>10}  {:>10}  {:>10}'.format(test_texts[i][j], test_tags[i][j], true_test_predictions[i][j]))

      Text       Label  Prediction
      aber           n           n
      auch           n           n
arschgeige         B-B         B-B
    trifft           n           n
        es           n           n
        ja           n           n
     nicht           n           n
  vollends           n           n
         .           n           n


In [ ]:
def computeTermEvalMetrics(extracted_terms, gold_df):
  extracted_terms = set([item.lower() for item in extracted_terms])
  gold_set=set(gold_df)
  true_pos=extracted_terms.intersection(gold_set)
  recall=len(true_pos)/len(gold_set)
  precision=len(true_pos)/len(extracted_terms)

  print("Intersection",len(true_pos))
  print("Gold",len(gold_set))
  print("Extracted",len(extracted_terms))
  print("Recall:", recall)
  print("Precision:", precision)
  print("F1:", 2*(precision*recall)/(precision+recall))

In [ ]:
test_extracted_terms = extract_terms(true_test_predictions, test_texts)

In [ ]:
computeTermEvalMetrics(test_extracted_terms, gold_set_for_test)

Intersection 135
Gold 144
Extracted 151
Recall: 0.9375
Precision: 0.8940397350993378
F1: 0.9152542372881357
